In [ ]:
from sciencecrawler.pubmed import PubmedSearch
import json
from neo4jsciencecrawler import Neo4jScienceCrawler
import time

In [ ]:
searchengine = PubmedSearch('covid chloroquine')

In [ ]:
total = searchengine.total_search

In [ ]:
total

In [ ]:
articles_current_page = searchengine.get_list_articles()

In [ ]:
neo4jC = Neo4jScienceCrawler("bolt://localhost:7687",'neo4j','admin')

In [ ]:
for article in articles_current_page:
    article_json = article.to_json()
    neo4jC.create_article(article_json)
    '''
    references = article.get_references()
    for ref in references:
        ref_json = ref.to_json()
        neo4jC.create_connection_ref(article_json['id'],ref_json)
    '''

In [ ]:
while articles_read < total:
    time.sleep(5)
    print(articles_read)
    searchengine.get_next_page()
    articles_current_page = searchengine.get_list_articles()
    articles_read = articles_read + len(articles_current_page)
    for article in articles_current_page:
        try:
            article_json = article.to_json()
            article_json['id'] = article_json['doi'].replace("doi:","").replace("/","").replace(".","").lower()
            neo4jC.create_article(article_json)
        except Exception as ex:
            print("=======================")
            print(article.title)
            print("\n")
            print(ex)
            print("====================================")